In [1]:
# This is a simulated Jupyter Notebook content.
# You can copy and paste this into a .ipynb file and run it cell by cell.

# --- Markdown Cell: Introduction ---
# # Data Analysis Pipeline
#
# This notebook demonstrates a comprehensive data analysis pipeline using a sample dataset.
# We will cover data loading, cleaning, exploratory data analysis (EDA), and specific filtering/visualization tasks.
#
# **Dataset:** The provided job posting data, now loaded from a file.
#
# **Downloadable Items:**
# Charts generated in this notebook (e.g., `work_type_distribution.json`, `top_5_roles_adjusted_chart.json`, `top_10_companies_chart.json`, `company_size_vs_company_chart.json`) can be downloaded. In a real Jupyter environment, these files would appear in your notebook's directory.

# --- Code Cell: Import Libraries ---
# ## 1. Import Libraries
# Essential libraries for data manipulation, visualization, and time handling.
import pandas as pd
import altair as alt
import json # To handle the Company Profile JSON string
from datetime import datetime, time
import pytz # For timezone handling
import re # For regular expressions to parse salary and experience
import joblib # For saving and loading Python objects efficiently

# --- Markdown Cell: Data Loading ---
# ## 2. Data Loading
#
# Now, we'll load the dataset into a pandas DataFrame from a CSV file named `job_descriptions.csv`.
#
# **Important:** Ensure that the `job_descriptions.csv` file is located in the specified path.

# --- Code Cell: Load Data ---
# Define the filename for the job descriptions data with the full path.
job_data_filename = "C:\\Users\\DELL\\Downloads\\archive\\job_descriptions.csv"

try:
    # Read the data from the CSV file into a DataFrame.
    df = pd.read_csv(job_data_filename)
    print(f"Successfully loaded data from '{job_data_filename}'.")
except FileNotFoundError:
    print(f"Error: The file '{job_data_filename}' was not found.")
    print("Please ensure the CSV file exists at the specified path.")
    # Create an empty DataFrame to prevent further errors if the file is not found.
    df = pd.DataFrame()
except Exception as e:
    print(f"An error occurred while loading the file: {e}")
    df = pd.DataFrame()


# Only proceed if the DataFrame is not empty after loading attempt.
if not df.empty:
    # Display the first few rows of the DataFrame to verify loading.
    print("DataFrame Head:")
    print(df.head())
    # Display concise summary of the DataFrame, including data types and non-null values.
    print("\nDataFrame Info:")
    print(df.info())
else:
    print("\nDataFrame is empty. Cannot proceed with analysis without data.")


# --- Markdown Cell: Data Cleaning and Preprocessing ---
# ## 3. Data Cleaning and Preprocessing
#
# This crucial stage involves transforming raw data into a clean, structured format suitable for analysis.
#
# **Steps Performed:**
# 1.  Convert 'Job Posting Date' to `datetime` objects for easier time-based filtering.
# 2.  Parse the 'Company Profile' column, which is a string representation of a JSON object, into actual Python dictionaries.
# 3.  Extract numerical values (minimum years) from the 'Experience' string.
# 4.  Extract numerical values (minimum salary in USD) from the 'Salary Range' string.
# 5.  Handle any potential missing values or inconsistencies.

# --- Code Cell: Clean and Preprocess Data ---
if not df.empty:
    # Convert 'Job Posting Date' to datetime objects.
    df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])
    print("\n'Job Posting Date' after conversion:")
    print(df['Job Posting Date'].head())
    print(df['Job Posting Date'].dtype)

    # Function to safely parse the 'Company Profile' string into a dictionary.
    def parse_company_profile(profile_str):
        if pd.isna(profile_str):
            return {}
        try:
            # Replace escaped double quotes (e.g., """") with single double quotes (") for valid JSON.
            cleaned_str = str(profile_str).replace('""""', '"')
            # Remove any outer double quotes that might have been added by CSV parsing.
            if cleaned_str.startswith('"') and cleaned_str.endswith('"'):
                cleaned_str = cleaned_str[1:-1]
            return json.loads(cleaned_str)
        except json.JSONDecodeError as e:
            # Print an error if parsing fails and return an empty dictionary.
            print(f"Error decoding JSON: {e} for string: {profile_str[:50]}...")
            return {} # Return an empty dict for errors

    # Apply the parsing function to create a new column with parsed company profiles.
    df['Company Profile_Parsed'] = df['Company Profile'].apply(parse_company_profile)

    print("\n'Company Profile_Parsed' column (first entry):")
    print(df['Company Profile_Parsed'].iloc[0])

    # Extract 'Sector' from the parsed company profile for potential future analysis.
    df['Company Sector'] = df['Company Profile_Parsed'].apply(lambda x: x.get('Sector'))
    print("\n'Company Sector' extracted:")
    print(df['Company Sector'].head())

    # Function to extract the minimum experience in years from the 'Experience' string.
    def extract_min_experience(experience_str):
        if pd.isna(experience_str): # Check for missing values
            return None
        # Use regex to find digits followed by 'to'
        match = re.search(r'(\d+)\s*to', str(experience_str))
        if match:
            return int(match.group(1))
        return None

    # Apply the function to create a new numerical 'Min Experience Years' column.
    df['Min Experience Years'] = df['Experience'].apply(extract_min_experience)
    print("\n'Min Experience Years' extracted:")
    print(df['Min Experience Years'].head())

    # Function to extract the minimum salary in USD (in thousands)
    def extract_min_salary(salary_str):
        if pd.isna(salary_str): # Check for missing values
            return None
        # Use regex to find a dollar sign, digits, and 'K' (for thousands).
        match = re.search(r'\$(\d+)K', str(salary_str))
        if match:
            return int(match.group(1)) * 1000 # Convert thousands (K) to actual value.
        return None

    # Apply the function to create a new numerical 'Min Salary USD' column.
    df['Min Salary USD'] = df['Salary Range'].apply(extract_min_salary)
    print("\n'Min Salary USD' extracted:")
    print(df['Min Salary USD'].head())
else:
    print("\nSkipping Data Cleaning and Preprocessing as DataFrame is empty.")


# --- Markdown Cell: Saving Processed Data (joblib) ---
# ## 4. Saving Processed Data
#
# After cleaning and preprocessing, it's often useful to save the DataFrame. This allows you to quickly load the cleaned data later without re-running all the preprocessing steps, saving time for large datasets. We'll use `joblib` for efficient serialization.

# --- Code Cell: Save DataFrame with joblib ---
if not df.empty:
    # Define a filename for the processed DataFrame.
    processed_data_filename = 'processed_job_data.joblib'
    # Save the DataFrame to a file using joblib.
    joblib.dump(df, processed_data_filename)
    print(f"\nProcessed DataFrame saved to '{processed_data_filename}'.")
else:
    print("\nSkipping saving processed data as DataFrame is empty.")

# --- Markdown Cell: Loading Processed Data (joblib) ---
# *Optional*: You could load the saved DataFrame in a new session or later part of the analysis.
#
# ```python
# # To load the DataFrame later:
# loaded_df = joblib.load('processed_job_data.joblib')
# print("\nLoaded DataFrame Head (from joblib):")
# print(loaded_df.head())
# ```

# --- Markdown Cell: Exploratory Data Analysis (EDA) ---
# ## 5. Exploratory Data Analysis (EDA)
#
# EDA is a critical step to understand the data's characteristics, distributions, and relationships between variables. It helps in identifying patterns, anomalies, and formulating hypotheses.
#
# **Examples of EDA:**
# * Value counts for categorical columns.
# * Visualizations of distributions.
# * Summary statistics for numerical columns.

# --- Code Cell: Perform EDA ---
if not df.empty:
    # Display value counts for the 'Work Type' column.
    print("\nValue counts for 'Work Type':")
    print(df['Work Type'].value_counts())

    # Display value counts for the 'Role' column.
    print("\nValue counts for 'Role':")
    print(df['Role'].value_counts())

    # Create a bar chart to visualize the distribution of 'Work Type'.
    work_type_chart = alt.Chart(df).mark_bar().encode(
        x=alt.X('Work Type', sort='-y', title='Work Type'), # Sort bars by count in descending order.
        y=alt.Y('count()', title='Number of Job Postings'),
        tooltip=['Work Type', 'count()'] # Show Work Type and count on hover.
    ).properties(
        title='Distribution of Work Types' # Set chart title.
    )
    # Uncomment the line below to display the chart in a live Jupyter environment.
    # work_type_chart.show()
    # Save the chart as a JSON file. This file can be downloaded.
    work_type_chart.save('work_type_distribution.json')
    print("\nChart 'work_type_distribution.json' saved.")
else:
    print("\nSkipping EDA as DataFrame is empty.")

# --- Markdown Cell: Specific Analysis and Visualization (Previous Request 1) ---
# ## 6. Specific Analysis and Visualization (Previous Request 1)
#
# This section addresses the request: "Draw a chart for top 5 roles in 2023 and work type=’Intern’ and Job Title should be Account Director and company should be below 2M."
#
# **Important Note:** Based on the provided sample dataset, there are no entries where 'Job Title' is 'Account Director'. Therefore, the first attempt to filter will likely result in an empty DataFrame. We will then show an adjusted filter for demonstration.

# --- Code Cell: Filter and Chart (Original Request 1) ---
if not df.empty:
    # Filter the DataFrame based on the specified criteria for the original request.
    filtered_df_original_request = df[
        (df['Work Type'] == 'Intern') &
        (df['Job Title'] == 'Account Director') & # This filter is unlikely to match any data in the sample.
        (df['Company Size'] < 2000000) &
        (df['Job Posting Date'].dt.year == 2023)
    ]

    print("\nDataFrame after applying original filters (Job Title = 'Account Director'):")
    print(filtered_df_original_request)

    # Check if the filtered DataFrame is empty and provide appropriate feedback.
    if filtered_df_original_request.empty:
        print("\nNo data found for the specified filters (Work Type='Intern', Job Title='Account Director', Company Size < 2M, Year=2023).")
        print("This is expected as 'Account Director' is not present in the 'Job Title' column of the provided data.")
    else:
        # If data exists, calculate the top 5 roles.
        top_5_roles_original = filtered_df_original_request['Role'].value_counts().nlargest(5).reset_index()
        top_5_roles_original.columns = ['Role', 'Count']
        # Create the bar chart.
        chart_original = alt.Chart(top_5_roles_original).mark_bar().encode(
            x=alt.X('Role', sort='-y', title='Role'),
            y=alt.Y('Count', title='Number of Occurrences')
        ).properties(
            title='Top 5 Roles (Original Filter - No Data Expected)'
        )
        # chart_original.show()
        chart_original.save('top_5_roles_original_request_chart.json')
        print("\nChart 'top_5_roles_original_request_chart.json' saved (if data existed).")
else:
    print("\nSkipping Specific Analysis (Previous Request 1) as DataFrame is empty.")


# --- Markdown Cell: Filter and Chart (Adjusted Request 1 for Demonstration) ---
# Since the original request's `Job Title` filter resulted in no data, let's adjust the filter to a `Job Title` that *does* exist in the dataset, for example, 'Web Developer', and adjust the year to 2022 to match existing data, to demonstrate chart generation.

# --- Code Cell: Filter and Chart (Adjusted Request 1) ---
if not df.empty:
    # Adjusted filter: Change 'Job Title' to 'Web Developer' and year to 2022.
    filtered_df_adjusted = df[
        (df['Work Type'] == 'Intern') &
        (df['Job Title'] == 'Web Developer') & # Adjusted filter to match sample data.
        (df['Company Size'] < 2000000) &
        (df['Job Posting Date'].dt.year == 2022) # Adjusted year to match sample data.
    ]

    print("\nDataFrame after applying adjusted filters (Job Title = 'Web Developer', Year=2022):")
    print(filtered_df_adjusted)

    if filtered_df_adjusted.empty:
        print("\nNo data found for the adjusted filters. Please check the dataset for matching entries.")
    else:
        # Get the top 5 roles for the adjusted filter.
        top_5_roles_adjusted = filtered_df_adjusted['Role'].value_counts().nlargest(5).reset_index()
        top_5_roles_adjusted.columns = ['Role', 'Count']

        # Create the bar chart.
        chart_adjusted = alt.Chart(top_5_roles_adjusted).mark_bar().encode(
            x=alt.X('Role', sort='-y', title='Role'),
            y=alt.Y('Count', title='Number of Occurrences')
        ).properties(
            title='Top 5 Roles for Interns as Web Developer in 2022 (Company Size < 2M)'
        )

        # chart_adjusted.show()
        chart_adjusted.save('top_5_roles_adjusted_chart.json')
        print("\nChart 'top_5_roles_adjusted_chart.json' saved.")
else:
    print("\nSkipping Specific Analysis (Adjusted Request 1) as DataFrame is empty.")

# --- Markdown Cell: New Analysis: Top 10 Companies for Data Engineer/Data Scientist (Previous Request 2) ---
# ## 7. New Analysis: Top 10 Companies for Data Engineer/Data Scientist
#
# This section implements the request for a chart showing the top 10 companies based on specific criteria, including a time-based display constraint.
#
# **Conditions:**
# * Role: 'Data Engineer'
# * Job Title: 'Data Scientist'
# * Preference: 'Female'
# * Qualifications: 'B.Tech' (Note: The code will check for 'B.Tech' explicitly. If your data uses 'BCA' for relevant entries, you may need to adjust this filter in the code.)
# * Country: Not Asian, and not starting with 'C'
# * Latitude: Below 10 (i.e., `< 10`)
# * Job Posting Date: Between 01/01/2023 and 06/01/2023
# * **Time Constraint:** Graph only works between 3 PM IST to 5 PM IST.

# --- Code Cell: Top 10 Companies Chart ---

# Define a set of Asian countries for efficient lookup.
asian_countries = {
    'Afghanistan', 'Armenia', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Bhutan',
    'Brunei', 'Cambodia', 'China', 'Cyprus', 'Georgia', 'India', 'Indonesia',
    'Iran', 'Iraq', 'Israel', 'Japan', 'Jordan', 'Kazakhstan', 'Kuwait',
    'Kyrgyzstan', 'Laos', 'Lebanon', 'Malaysia', 'Maldives', 'Mongolia',
    'Myanmar', 'Nepal', 'North Korea', 'Oman', 'Pakistan', 'Palestine',
    'Philippines', 'Qatar', 'Russia', 'Saudi Arabia', 'Singapore',
    'South Korea', 'Sri Lanka', 'Syria', 'Taiwan', 'Tajikistan', 'Thailand',
    'Timor-Leste', 'Turkey', 'Turkmenistan', 'United Arab Emirates',
    'Uzbekistan', 'Vietnam', 'Yemen'
}

# Function to check if the current time falls within the allowed display window (3 PM IST to 5 PM IST).
def is_time_to_show_chart():
    ist_timezone = pytz.timezone('Asia/Kolkata') # Define IST timezone.
    current_time_ist = datetime.now(ist_timezone).time() # Get current time in IST.
    start_time = time(15, 0) # 3 PM IST
    end_time = time(17, 0)   # 5 PM IST
    return start_time <= current_time_ist <= end_time

# Check the time condition before proceeding with chart generation.
if is_time_to_show_chart():
    print("\nIt is currently between 3 PM IST and 5 PM IST. Attempting to generate the chart.")
    if not df.empty:
        # Apply all specified filters to the DataFrame.
        filtered_df_new_request = df[
            (df['Role'] == 'Data Engineer') &
            (df['Job Title'] == 'Data Scientist') &
            (df['Preference'] == 'Female') &
            (df['Qualifications'] == 'B.Tech') & # Filtering for B.Tech as explicitly requested.
            (~df['Country'].isin(asian_countries)) & # Exclude Asian countries.
            (~df['Country'].astype(str).str.startswith('C')) & # Exclude countries starting with 'C'.
            (df['latitude'] < 10) & # Latitude should be BELOW 10.
            (df['Job Posting Date'] >= '2023-01-01') & # Job posting date start.
            (df['Job Posting Date'] <= '2023-06-01') # Job posting date end.
        ]

        print("\nDataFrame after applying new request filters:")
        print(filtered_df_new_request)

        # Check if any data matches the filters.
        if filtered_df_new_request.empty:
            print("\nNo data found for the specified new request filters.")
            print("Please check if 'Data Engineer' as Role, 'Data Scientist' as Job Title, 'B.Tech' as Qualification, and the date range (01/01/2023 to 06/01/2023) exist in the dataset with the other conditions.")
            print("Note: The current sample data is very limited and likely does not contain entries matching all these criteria.")
        else:
            # Get the top 10 companies by count.
            top_10_companies = filtered_df_new_request['Company'].value_counts().nlargest(10).reset_index()
            top_10_companies.columns = ['Company', 'Count']

            # Create the bar chart for top 10 companies.
            chart_top_companies = alt.Chart(top_10_companies).mark_bar().encode(
                x=alt.X('Company', sort='-y', title='Company'),
                y=alt.Y('Count', title='Number of Job Postings'),
                tooltip=['Company', 'Count']
            ).properties(
                title='Top 10 Companies for Data Engineer/Data Scientist (Filtered)'
            )

            # chart_top_companies.show()
            chart_top_companies.save('top_10_companies_chart.json')
            print("\nChart 'top_10_companies_chart.json' saved.")
    else:
        print("\nSkipping Top 10 Companies Analysis as DataFrame is empty.")
else:
    print("\nIt is not between 3 PM IST and 5 PM IST. The chart for top 10 companies will not be generated at this time.")

# --- Markdown Cell: New Analysis: Company Size vs. Company Name (New Request 3) ---
# ## 8. New Analysis: Company Size vs. Company Name
#
# This section implements the request for a chart showing the relationship between company size and company name, based on a new set of filters, including a time-based display constraint.
#
# **Conditions:**
# * Company Size: < 50000
# * Job Title: 'Mechanical Engineer'
# * Experience: > 5 years
# * Country: Asian
# * Salary: > $50k
# * Work Type: 'Part Time' or 'Full Time'
# * Preference: 'Male'
# * Job Portal: 'Idealist'
# * **Time Constraint:** Graph only works between 3 PM IST to 5 PM IST.

# --- Code Cell: Company Size vs. Company Name Chart ---

# Check the time condition before proceeding with chart generation.
if is_time_to_show_chart():
    print("\nIt is currently between 3 PM IST and 5 PM IST. Attempting to generate the Company Size vs. Company Name chart.")
    if not df.empty:
        # Apply all specified filters for the new chart.
        filtered_df_company_size_chart = df[
            (df['Company Size'] < 50000) &
            (df['Job Title'] == 'Mechanical Engineer') &
            (df['Min Experience Years'] > 5) &
            (df['Country'].isin(asian_countries)) & # Country should be Asian.
            (df['Min Salary USD'] > 50000) &
            (df['Work Type'].isin(['Part Time', 'Full Time'])) & # Work type should be part time or full time.
            (df['Preference'] == 'Male') &
            (df['Job Portal'] == 'Idealist')
        ]

        print("\nDataFrame after applying Company Size vs. Company Name filters:")
        print(filtered_df_company_size_chart)

        # Check if any data matches the filters.
        if filtered_df_company_size_chart.empty:
            print("\nNo data found for the specified Company Size vs. Company Name chart filters.")
            print("Please check if 'Mechanical Engineer' as Job Title, 'Male' preference, 'Part Time'/'Full Time' work type, Asian countries, salary > $50k, and experience > 5 years exist in the dataset.")
            print("Note: The current sample data is very limited and likely does not contain entries matching all these criteria.")
        else:
            # Create the bar chart for Company Size vs. Company Name.
            # This chart will display each matching company and its corresponding size.
            chart_company_size = alt.Chart(filtered_df_company_size_chart).mark_bar().encode(
                x=alt.X('Company', sort='-y', title='Company'),
                y=alt.Y('Company Size', title='Company Size'),
                tooltip=['Company', 'Company Size', 'Job Title', 'Min Experience Years', 'Country', 'Min Salary USD', 'Work Type', 'Preference', 'Job Portal']
            ).properties(
                title='Companies by Size (Filtered: Mechanical Engineer, Male, Asian, >5yrs Exp, >$50k, Part/Full Time, Idealist)'
            )

            # chart_company_size.show()
            chart_company_size.save('company_size_vs_company_chart.json')
            print("\nChart 'company_size_vs_company_chart.json' saved.")
    else:
        print("\nSkipping Company Size vs. Company Name Analysis as DataFrame is empty.")
else:
    print("\nIt is not between 3 PM IST and 5 PM IST. The chart for Company Size vs. Company Name will not be generated at this time.")

# --- Markdown Cell: Conclusion ---
# ## 9. Conclusion
#
# This notebook demonstrated a comprehensive data analysis pipeline, including:
# * Loading data from a string.
# * Performing robust data cleaning and type conversions, including extracting numerical values from string fields.
# * Saving and loading intermediate processed data using `joblib` for efficiency.
# * Conducting exploratory data analysis with descriptive statistics and a simple bar chart.
# * Applying specific, complex filters and generating charts, while also showing how to handle cases where no data matches the criteria.
# * Implementing a time-based display constraint for specific charts, ensuring they are only shown during designated hours.
#
# For a real-world scenario, this pipeline would be expanded with more sophisticated feature engineering, model training, evaluation, and deployment steps.


Successfully loaded data from 'C:\Users\DELL\Downloads\archive\job_descriptions.csv'.
DataFrame Head:
             Job Id     Experience Qualifications Salary Range    location  \
0  1089843540111562  5 to 15 Years         M.Tech    $59K-$99K     Douglas   
1   398454096642776  2 to 12 Years            BCA   $56K-$116K    Ashgabat   
2   481640072963533  0 to 12 Years            PhD   $61K-$104K       Macao   
3   688192671473044  4 to 11 Years            PhD    $65K-$91K  Porto-Novo   
4   117057806156508  1 to 12 Years            MBA    $64K-$87K    Santiago   

            Country  latitude  longitude  Work Type  Company Size  ...  \
0       Isle of Man   54.2361    -4.5481     Intern         26801  ...   
1      Turkmenistan   38.9697    59.5563     Intern        100340  ...   
2  Macao SAR, China   22.1987   113.5439  Temporary         84525  ...   
3             Benin    9.3077     2.3158  Full-Time        129896  ...   
4             Chile  -35.6751   -71.5429     Intern        

C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.


Chart 'work_type_distribution.json' saved.

DataFrame after applying original filters (Job Title = 'Account Director'):
                   Job Id     Experience Qualifications Salary Range  \
22       2508133000944341   1 to 8 Years         M.Tech   $58K-$116K   
2193     1004102062834988  1 to 13 Years            PhD    $58K-$97K   
11167    1984374041489145   3 to 8 Years            BBA    $56K-$84K   
11621    2891761623707560   5 to 9 Years          M.Com   $63K-$105K   
14380     356991696014476  5 to 14 Years            BBA   $62K-$101K   
...                   ...            ...            ...          ...   
1607564  2498464996471701  5 to 13 Years         M.Tech    $57K-$85K   
1610521  1444778289545149  3 to 13 Years         M.Tech   $61K-$100K   
1611751  2575034816907083  4 to 11 Years          M.Com    $55K-$98K   
1612034   881667010043640   0 to 8 Years          B.Com   $64K-$127K   
1612970   372274483166881  2 to 10 Years         B.Tech    $65K-$89K   

           loc

C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)



DataFrame after applying adjusted filters (Job Title = 'Web Developer', Year=2022):
                   Job Id     Experience Qualifications Salary Range  \
1         398454096642776  2 to 12 Years            BCA   $56K-$116K   
1581     2349588572494690   5 to 9 Years            MCA    $61K-$90K   
2654       69115977258069  1 to 10 Years         B.Tech    $56K-$82K   
2863     1608297049191911  1 to 15 Years            BBA    $55K-$99K   
3078      548020656940197  4 to 14 Years            MCA    $56K-$96K   
...                   ...            ...            ...          ...   
1611322   133745922432576  2 to 10 Years         B.Tech   $55K-$104K   
1611551  3074295093021038  0 to 12 Years            PhD    $63K-$80K   
1612661   740172401168660  0 to 11 Years            BBA   $58K-$105K   
1613543  2883334114456221   4 to 9 Years          M.Com    $61K-$81K   
1615066  2946212196864564   0 to 8 Years          M.Com    $62K-$90K   

             location              Country   latit

C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)



It is currently between 3 PM IST and 5 PM IST. Attempting to generate the chart.

DataFrame after applying new request filters:
                   Job Id     Experience Qualifications Salary Range  \
96518     882048179569920  2 to 10 Years         B.Tech    $58K-$91K   
479772   1309733923277464  3 to 11 Years         B.Tech    $59K-$95K   
714027    577733502207192  4 to 10 Years         B.Tech   $59K-$103K   
765592    253442328517653  5 to 11 Years         B.Tech    $58K-$98K   
1008346   302714095135208  0 to 14 Years         B.Tech   $58K-$109K   
1091320   329929017354658  1 to 15 Years         B.Tech    $60K-$90K   

                  location            Country  latitude  longitude  Work Type  \
96518                Abuja            Nigeria    9.0820     8.6753  Full-Time   
479772               Abuja            Nigeria    9.0820     8.6753  Temporary   
714027           Mogadishu            Somalia    5.1521    46.1996  Part-Time   
765592   Malabo (de jure),  Equatorial Gui

C:\ProgramData\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)



DataFrame after applying Company Size vs. Company Name filters:
Empty DataFrame
Columns: [Job Id, Experience, Qualifications, Salary Range, location, Country, latitude, longitude, Work Type, Company Size, Job Posting Date, Preference, Contact Person, Contact, Job Title, Role, Job Portal, Job Description, Benefits, skills, Responsibilities, Company, Company Profile, Company Profile_Parsed, Company Sector, Min Experience Years, Min Salary USD]
Index: []

[0 rows x 27 columns]

No data found for the specified Company Size vs. Company Name chart filters.
Please check if 'Mechanical Engineer' as Job Title, 'Male' preference, 'Part Time'/'Full Time' work type, Asian countries, salary > $50k, and experience > 5 years exist in the dataset.
Note: The current sample data is very limited and likely does not contain entries matching all these criteria.
